In [7]:
from audiotsm import phasevocoder
from audiotsm.io.wav import WavReader, WavWriter
import os
import numpy as np
from scipy.io.wavfile import read
from scipy import signal, fft
import matplotlib.pyplot as plt
import cv2
import librosa


AssertionError: 

In [ ]:
# Input and plot waveform
fs, audio_arr = read("./audio/output/piano.wav_scaled_0.5.wav")
length = audio_arr.shape[0]/fs
time = np.linspace(0., length, audio_arr.shape[0])

plt.figure(1)
plt.title("Signal Wave")
plt.plot(time, audio_arr[:, 0], label="Left channel")
plt.plot(time, audio_arr[:, 1], label="Right channel")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.show()

plt.figure(2)
plt.phase_spectrum(audio_arr[:, 0], color="red")
plt.phase_spectrum(audio_arr[:, 1], color="green")
plt.title("Phase Spectrum of the signal")
plt.show()

In [ ]:
#Filtering Audio
filt = signal.butter(3, 0.5, output='sos')
audio_filtered = signal.sosfilt(filt, audio_arr)
print(audio_arr.shape[0])

plt.figure(3)
plt.title("Filtered Signal Wave...")
plt.plot(audio_filtered)
plt.show()

plt.figure(4)
plt.phase_spectrum(audio_filtered[:, 0], color="red")
plt.phase_spectrum(audio_filtered[:, 1], color="green")
plt.title("Phase Spectrum of the filtered signal")
plt.show()

#Spettrogramma
#controllare phasevocoder audiotsm se c'è spettrogramma

In [ ]:
block = 256     # cropped block size
lam = 1         # contrast parameter
sig = 1         # contrast parameter
tau = 2         # contrast parameter
dc_margin = 10  # zero-region around DC

# FFT contrast function
def contrast(x):
    return cv2.GaussianBlur(x, (3, 3), 0)**3

# P-map computation
spectrum = fft(audio_arr[:,0])
magnitude = np.abs(spectrum)
phase = np.angle(spectrum)


e = audio_arr - audio_filtered
p = lam * np.exp(-(np.abs(e)**tau) / sig)


P = contrast(np.abs(np.fft.fft2(p)))

plt.figure()
plt.title("Error amplitude")
plt.plot(time, e[:, 0], label="Left channel")
plt.plot(time, e[:, 1], label="Right channel")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.show()

plt.figure()
plt.phase_spectrum(e[:, 0], color="red")
plt.phase_spectrum(e[:, 1], color="green")
plt.title("Phase Spectrum of the error")
plt.show()

plt.figure()
plt.plot(phase)
plt.title("PHASE")

plt.figure()
plt.title("p amplitude")
plt.plot(time, p[:, 0], label="Left channel")
plt.plot(time, p[:, 1], label="Right channel")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.show()

plt.figure()
plt.phase_spectrum(p[:, 0], color="red")
plt.phase_spectrum(p[:, 1], color="green")
plt.title("Phase Spectrum of the p signal")
plt.show()




In [ ]:
#shift back signal and correlation
audio_shift = audio_arr[1, -1]
audio_short = audio_arr[0, -2]
corr = signal.correlate((audio_shift[:, 1]), (audio_short[:, 1]))

plt.figure()
plt.plot(corr)
plt.title("Correlation")




In [ ]:
# Cumulative periodogram
num = np.cumsum(np.cumsum(np.abs(P)**2, axis=1), axis=0)
den = np.sum(np.abs(P)**2)
print(den)
C = num / den

In [ ]:
# Remove DC peaks
C = C[dc_margin:, dc_margin:]
print("ok")

In [ ]:
# Detect
sobelx = cv2.Sobel(C, cv2.CV_64F, 1, 0, ksize=5)
sobely = cv2.Sobel(C, cv2.CV_64F, 0, 1, ksize=5)
delta = np.max((np.abs(sobelx), np.abs(sobely)))
print("ok")

In [ ]:
# Display
print('Confidence: {}'.format(delta))
plt.figure(figsize=(10, 10))
plt.subplot(121), plt.imshow(P[dc_margin: , dc_margin:]), plt.title('processed pmap')
plt.subplot(122), plt.imshow(C[dc_margin: , dc_margin:]), plt.title('detection map')
plt.show()